In [20]:
import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np

In [53]:
base = cv.imread("base.png", cv.IMREAD_GRAYSCALE)

img = cv.imread("base_photo.png", cv.IMREAD_GRAYSCALE)

escala = 0.007843137
centro_base = (140,135)

K = [[1,0,0],[0,1,0],[0,0,1]]

In [12]:
def inverteTransformacao(R, t):
    RInverso = np.transpose(R)

    tInverso  = - np.matmul(RInverso, t)

    return RInverso, tInverso

In [54]:
orb = cv.ORB_create()

kp1, des1 = orb.detectAndCompute(base, None)
kp2, des2 = orb.detectAndCompute(img, None)

FLANN_INDEX_LSH = 6
index_params = dict(algorithm = FLANN_INDEX_LSH, table_number = 6, key_size = 12, multi_probe_level = 1)
search_params = dict(checks = 100)

flann = cv.FlannBasedMatcher(index_params, search_params)

matches = flann.knnMatch(des1,des2,k=2)

good = []
for i,m_n in enumerate(matches):
    if len(m_n) != 2:
        continue
    (m,n) = m_n
    if m.distance < 0.7*n.distance:
        good.append(m)

#Calcula os pontos da base na imagem
pontoImagem = []

pontoReal = []

#train = kp2
#query = kp1

for match in good:
    point_base = kp1[match.queryIdx].pt
    point_photo = kp2[match.trainIdx].pt

    #Desloca o ponto para o centro de coordenadas do centro da base
    point_baseMetro = [point_base[0]-centro_base[0],point_base[1]-centro_base[1], 0]

    #Converte para metro
    point_baseMetro[0] = point_baseMetro[0] * escala 
    point_baseMetro[1] = point_baseMetro[1] * escala 

    #Adiciona as listas
    pontoImagem.append([point_photo[0],point_photo[1]])
    pontoReal.append(point_baseMetro)

pontoImagem = np.array(pontoImagem, dtype=np.float32)
pontoReal = np.array(pontoReal,dtype=np.float32)
K = np.array(K,dtype=np.float32)

a, RObj, tObj = cv.solvePnP(pontoReal,pontoImagem , K, np.zeros((5,1)))

RObj, _ = cv.Rodrigues(RObj)

RCamera, tCamera = inverteTransformacao(RObj, tObj)

In [55]:
tCamera

array([[  9.10245563e-02],
       [ -2.35009786e-02],
       [ -1.36494586e-05]])

In [49]:
RObj

(array([[-0.23682134, -0.9715473 ,  0.00339186],
        [-0.97155104,  0.23682676,  0.00129102],
        [-0.00205757, -0.00298963, -0.99999341]]),
 array([[ -4.85602877e-01,   1.20066559e-01,   4.86227256e-01,
           1.19398679e-01,   4.87742971e-01,   3.80568933e-01,
          -4.86391693e-01,  -3.81235843e-01,   2.14055131e-03],
        [ -3.84175984e-01,   9.14868634e-02,  -6.18320665e-01,
           9.23371016e-02,   3.81451560e-01,  -4.86229127e-01,
           6.17653755e-01,   4.86389821e-01,  -2.72500703e-03],
        [  2.63806881e-04,  -1.43727072e-03,  -3.93265318e-01,
           7.70361154e-04,   4.26372227e-04,   5.01517537e-01,
          -3.94115556e-01,   5.00849656e-01,  -6.86437079e-04]]))